# Define most important functions

In [1]:
import cv2
import numpy as np
import os

def detect_vehicles_yolo_tiny_img(img, weights_path='yolov3-tiny.weights', cfg_path='yolov3-tiny.cfg', class_names_path='coco.names', confidence_threshold=0.5, nms_threshold=0.4):
  
    try:
        # Load the image
        height, width, channels = img.shape
        # Load class names
        with open(class_names_path, 'r') as f:
            classes = [line.strip() for line in f.readlines()]
        target_classes = ['bus', 'truck', 'motorbike','car']
        target_class_indices = []
        for class_name in target_classes:
            try:
                target_class_indices.append(classes.index(class_name))
            except ValueError:
                print(f"Warning: '{class_name}' class not found in the class names file.")
        if not target_class_indices:
            print("Error: None of the target vehicle classes  were found in the class names file.")
            return img, None
        # Load the YOLOv3-Tiny network
        net = cv2.dnn.readNet(weights_path, cfg_path)
        layer_names = net.getLayerNames()
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

        # Create a blob from the image
        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # Information for drawing bounding boxes
        colors = np.random.uniform(0, 255, size=(len(classes), 3))
        detections = []

        # Process the output layers
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > confidence_threshold and class_id in target_class_indices:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    detections.append([x, y, w, h, confidence, classes[class_id]])

        # Apply Non-Maximum Suppression (NMS)
        indices = cv2.dnn.NMSBoxes(
            [detection[:4] for detection in detections],
            [detection[4] for detection in detections],
            confidence_threshold,
            nms_threshold
        )

        detected_vehicles = []
        if len(indices) > 0:
            for i in indices.flatten():
                x, y, w, h, confidence, class_name = detections[i]
                color = [255,0,0]#colors[classes.index(class_name)]
                # Draw the bounding box
                """                
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                # Add the label and confidence
                cv2.putText(img, f"{class_name} {confidence:.2f}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)"""
                detected_vehicles.append((x, y, w, h, confidence, class_name))

        if detected_vehicles:
            return img, detected_vehicles
        else:
            print("No buses, trucks, or motorbikes detected in the image.")
            return img, None

    except FileNotFoundError as e:
        print(f"Error: One or more required files not found: {e}")
        return None, None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

In [11]:
def process_detections_with_quadrangle(quad_points, img, detections,overlap_percentage_threshold):
    wrongCars=0
    if img is None:
        return None

    if detections is None:
        cv2.polylines(img, [np.array(quad_points, np.int32).reshape((-1, 1, 2))], True, (0, 0, 0), 2)
        return img

    # Draw the quadrangle border
    cv2.polylines(img, [np.array(quad_points, np.int32).reshape((-1, 1, 2))], True, (0, 0, 0), 2)

    for detection in detections:
        x, y, w, h, confidence, class_name = detection
        x_min, y_min, x_max, y_max = x, y, x + w, y + h
        bbox_points = [(x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max)]

        # Calculate the percentage of the bounding box inside the quadrangle
        inside_count = 0
        total_points = len(bbox_points)

        quad_polygon = np.array(quad_points, np.int32).reshape((1, -1, 2))

        for point in bbox_points:
            if cv2.pointPolygonTest(quad_polygon, point, False) >= 0:
                inside_count += 1

        if total_points > 0:
            overlap_percentage = (inside_count / total_points) * 100
        else:
            overlap_percentage = 0

        color = (255, 0, 0)  # Default blue color
        if overlap_percentage >= overlap_percentage_threshold*100:
            color = (0, 0, 255)  # Red color
            wrongCars+=1
            

        # Draw the bounding box
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        # Add the label and confidence
        cv2.putText(img, f"{class_name} {confidence:.2f}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return img,wrongCars

## Let's play with them

get the datafiles for neural network

In [3]:
weights_file = "/home/zsicomp/Downloads/darknet-test-master/yolov3-tiny.weights"  # Path to your weights file
config_file = "/home/zsicomp/Downloads/darknet-test-master/yolov3-tiny.cfg"     # Path to your configuration file
class_names_file = "/home/zsicomp/Downloads/darknet-test-master/coco.names"   # Path to the class names file

put our image file path here:

In [16]:
#image_file = "/home/zsicomp/Downloads/20250304_131409.jpg"  #My photo
#image_file ="/home/zsicomp/Downloads/ÜllőiÚt2024.png" #No bad car on it rossz auto rajta
image_file="/home/zsicomp/Downloads/BiciklisfutarAutotKerul.jpg"
#image_file = "/home/zsicomp/Downloads/UlloiBringasavParkolas.png"  # Replace with the path to your image
#image_file = "/home/zsicomp/Downloads/UlloiUtParkolas2.png" 

In [17]:
image = cv2.imread(image_file) # load the chosen image

Resize the image if it is too large for better handling (for computer vision we will use anyway images around 400 px)

In [18]:
#reshape image if it is too large
print(image.shape,"Original Image size")

if image.shape[0]>700 or image.shape[1]>700:
    if image.shape[0] < image.shape[1]:
        scale_factor= 700/image.shape[1]
    else:
        scale_factor= 700/image.shape[0]
    width = int(image.shape[1] * scale_factor)
    height = int(image.shape[0] * scale_factor)
    dim = (width, height)
    img = cv2.resize(image[:,:,:], dim, interpolation=cv2.INTER_AREA)
else:
    img=image
    
print(img.shape,"Resized image")
# Resize the image

#print(resized_img.shape)

(3024, 4032, 3) Original Image size
(525, 700, 3) Resized image


### Select the area within the cars are in wrong place. Click around the area. After each click you will see a circle and a number where you clicked. you can slect only one patch, and make it **convex**;
## After clicking exit with any button

In [ ]:
## Select the

In [19]:
#find the points within them we think cars must not park.
# click on points in an order, when you connect them get a good looking rectangle
PointPairs=[]
imgForPints=img.copy()
def draw_circle(event, x, y, flags, param):
    
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img=imgForPints, center=(x,y), radius=10, color=(0,255,0), thickness=5)
        PointPairs.append((x,y))
        cv2.putText(imgForPints, str(len(PointPairs)),(x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, [0,0,240], 2) # we will nummber the clicks

# conect drawing function to image window
cv2.namedWindow(winname='image')
cv2.setMouseCallback('image', draw_circle)

# showing image

while True:
    
    cv2.imshow('image', imgForPints)
    
    if cv2.waitKey(20) & 0xFF == 27:
        break

cv2.destroyAllWindows()

![image info](./RosszTartomanyKijelolese.png)

Do the detection:

In [20]:
detected_image, vehicle_detections = detect_vehicles_yolo_tiny_img(
    img,
    weights_path=weights_file,
    cfg_path=config_file,
    class_names_path=class_names_file,
    confidence_threshold=0.2,
    nms_threshold=0.3
)# make the image detection, with relatively low confidence values

## Show recognised image
* Around the forbidden are you will have a black line
* "Cars" in wrong place get red rectangle around them
* "Cars" detected on other places will get blue rectangles

In [21]:
EpicImage4,wrongCars=process_detections_with_quadrangle(PointPairs, img, vehicle_detections,0.4)
cv2.imshow("Vehicle Detection", EpicImage4)
cv2.waitKey(0)
cv2.destroyAllWindows()

![image info](./FinallyCarIsInWrongPlace.png)

In [22]:
if wrongCars==0:
    print("There are only good drivers, or computer vision is not working well")
else:
    print("Number of cars in wrong place are:", wrongCars)

Number of cars in wrong place are: 1


## Later I plan to recognise car color and numberplate, thereby I can tell the car owner on a loud speaker to move the car. And with number plate and car color maybe the message will be more personal